# Country Coordinates and TimeZones
With the saved countries dataset (my repo -> Python-Daily/Fun with Countries), we will fetch the lat-long coordinates and timezone for our list of countries.

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import io

# Fetching our dataset from my Github repo (using the bytesIO process for a numpy-file case)
url = 'https://github.com/FaarisRazi/Python-Daily/blob/main/Fun%20with%20Countries/countries_geo.npy'
response = requests.get(url+'?raw=true')
response.raise_for_status()
bytesIO_obj = io.BytesIO(response.content)

country_df = np.load(bytesIO_obj, allow_pickle = True).item()['df']
country_df = country_df[['iso_a3']] # Just keeping the ISO-codes column
country_df.head()

,iso_a3
country,
Aruba,ABW
Afghanistan,AFG
Angola,AGO
Anguilla,AIA
Albania,ALB


### Getting every Country's Coordinates
To geocode our country's addresses and obtain their lat-long coordinates, the **geocoder** API does the job!
*Source*: https://geocoder.readthedocs.io/api.html

Here's a test in fetching some random country's coordinates via ArcGIS in geocoder (Google is an option too).

In [3]:
from ttictoc import tic, toc # To track the time
from geocoder import arcgis
from random import choice

random_country = choice(country_df.index)

tic()
geo_obj =  arcgis(random_country)
coords = geo_obj.latlng

print(f"ArcGIS test for the Coordinates of {random_country} is:"+
      f"\nLatitude = {coords[0]}\nLongitude = {coords[1]}\n"+
      f"\n\nGeocoding time taken: {round(toc(), 3)} seconds.")

ArcGIS test for the Coordinates of Luxembourg is:
Latitude = 49.776828429000034
Longitude = 6.092325247000076


Geocoding time taken: 0.901 seconds.


To geocode **multiple addresses** as in our case, we need "persistent HTTP connection" with requests.Session() as per the geocoder's API docs (see *Using a Session* section).

In [3]:
country_coords = {}
with requests.Session() as session:
    
    for i, country in enumerate(country_df.index, 1):
        coords = arcgis(country, session=session).latlng
        
        country_coords[country] = coords
        
        # For tracking progress,
        if not i%20: # print every 20th country's coordinates.
            print(f"{country}'s:\t{coords}")

print("All {country_df.shape[0]} country's coordinates collected!")

Belgium's:	[50.640682937000065, 4.661070427000027]
Canada's:	[60.108670000000075, -113.64257999999995]
Cyprus's:	[35.05017420900003, 33.22622977900005]
Federated States of Micronesia's:	[6.880382388000044, 158.22751848500002]
Croatia's:	[45.11867958500005, 15.435623834000069]
Kazakhstan's:	[48.18310616400004, 67.19504548200007]
Macao S.A.R's:	[22.15778000000006, 113.55972000000008]
Malawi's:	[-13.50952235099993, 34.24073561300003]
Philippines's:	[14.164862797000069, 120.8616300000001]
Serranilla Bank's:	[15.84907000000004, -79.85961999999995]
Syria's:	[35.01280854600003, 38.50527333900004]
US Naval Base Guantanamo Bay's:	[20.48028000000005, -74.61693999999994]


{'Aruba': [12.50908537500004, -69.97050301699994],
 'Afghanistan': [33.831137065000064, 66.02471179700007],
 'Angola': [-12.293655625999975, 17.545335193000028],
 'Anguilla': [18.224602780000055, -63.05962295599994],
 'Albania': [41.13455328400005, 20.06420643100006],
 'Aland': [17.564130000000034, 76.56469000000004],
 'Andorra': [42.54530320100008, 1.5762863020000282],
 'United Arab Emirates': [23.914869621000037, 54.32685818400006],
 'Argentina': [-35.49575818399995, -65.07154210799996],
 'Armenia': [40.293084855000075, 44.94022131100007],
 'American Samoa': [-14.30068805999997, -170.71811612199997],
 'Antarctica': [-80.56228773918775, 20.797320000417756],
 'Ashmore and Cartier Islands': [-12.416669999999954, 123.33333000000005],
 'French Southern and Antarctic Lands': [-49.60647870499997,
  69.64303000000007],
 'Antigua and Barbuda': [17.077664637000055, -61.79871012099994],
 'Australia': [-25.70993156999998, 134.48403119800003],
 'Austria': [47.58581000500004, 14.137076948000072],


In [13]:
country_df[['Latitude','Longitude']] = pd.DataFrame(country_df.index.map(country_coords.get).tolist(),
                                                   index = country_df.index)

country_df

,iso_a3,"(Latitude, Longitude)",Latitude,Longitude
country,,,,
Aruba,ABW,"[12.50908537500004, -69.97050301699994]",12.509085,-69.970503
Afghanistan,AFG,"[33.831137065000064, 66.02471179700007]",33.831137,66.024712
Angola,AGO,"[-12.293655625999975, 17.545335193000028]",-12.293656,17.545335
Anguilla,AIA,"[18.224602780000055, -63.05962295599994]",18.224603,-63.059623
Albania,ALB,"[41.13455328400005, 20.06420643100006]",41.134553,20.064206
...,...,...,...,...
Samoa,WSM,"[-13.62080777999995, -172.44733321399997]",-13.620808,-172.447333
Yemen,YEM,"[15.905205994000028, 47.59395262500004]",15.905206,47.593953
South Africa,ZAF,"[-28.997182287999976, 25.085049940000033]",-28.997182,25.085050


### Getting every Country's TimeZone
Here's a function we will use to collect the TimeZones, with the *TimezoneFinder* package: (https://pypi.org/project/timezonefinder/).

In [14]:
from timezonefinder import TimezoneFinder

def gettz(coords):
    """
    Get the Timezone name of location coordinates via TimezoneFinder 
    API.

    Parameters
    ----------
    lat : float
        Latitude in degrees north
    lon : float
        Longitude in degrees east

    Returns
    -------
    Timezone: str
        TZ name of the location coordinates.
    """
   
    if type(coords) == str:
        coords = list(map(lambda x: float(x.strip()),coords.split(',')))

    lat, lon = coords

    tf = TimezoneFinder()

    try:
        timezone_name = tf.timezone_at(lng=lon, lat=lat)
        if timezone_name is None:
            timezone_name = tf.closest_timezone_at(lng=lon, lat=lat)

        return timezone_name
            # maybe even increase the search radius when it is still None
    except ValueError or NameError:
        # the coordinates were out of bounds
        pass # {handle error}

#### Timezones Collected:

In [19]:
country_df['timezone'] = country_df.apply(lambda x: gettz([x['Latitude'], x['Longitude']]), 
                 axis=1)

country_df

,iso_a3,"(Latitude, Longitude)",Latitude,Longitude,timezone
country,,,,,
Aruba,ABW,"[12.50908537500004, -69.97050301699994]",12.509085,-69.970503,America/Aruba
Afghanistan,AFG,"[33.831137065000064, 66.02471179700007]",33.831137,66.024712,Asia/Kabul
Angola,AGO,"[-12.293655625999975, 17.545335193000028]",-12.293656,17.545335,Africa/Luanda
Anguilla,AIA,"[18.224602780000055, -63.05962295599994]",18.224603,-63.059623,America/Anguilla
Albania,ALB,"[41.13455328400005, 20.06420643100006]",41.134553,20.064206,Europe/Tirane
...,...,...,...,...,...
Samoa,WSM,"[-13.62080777999995, -172.44733321399997]",-13.620808,-172.447333,Pacific/Apia
Yemen,YEM,"[15.905205994000028, 47.59395262500004]",15.905206,47.593953,Asia/Aden
South Africa,ZAF,"[-28.997182287999976, 25.085049940000033]",-28.997182,25.085050,Africa/Johannesburg
